# **Proyecto UAO - Piloto Deserción**
## Gerente UAO:  Erik Orozco
## Consultor Innovadata.ai:  Jaime Reinoso

Actores: 
EO. Erik Orozco
JR.  Jaime Reinoso

Formato fechas: YYYY.MM.DD

Hitos Proyecto:
2023.08.30 - Reunion de kickoff:  se explica alcance, proceso a seguir y data posible a capturar.  Powerpoint de soporte.
2023.09.08 - Se recibe email de ERIK con data
2023.09.11 - Jaime analiza data

**Importaciones Globales**


In [1]:
from pandas_profiling import ProfileReport
import pandas as pd

/Users/jaimereinoso/DESARROLLO/UAOPiloto/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_99273/3708703399.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


## 1.  Recolección de Datos
Como primera medida se leen los archivos entregados con el fin de entender la data, detectar inconsistencias y validar cantidad de filas disponibles.


### **1.1 Estudiantes**

In [6]:
from pandas_profiling import ProfileReport
import pandas as pd

estudiantes = pd.read_excel('data_original/estudiante.xlsx')
prof = ProfileReport(estudiantes)
prof.to_file(output_file='estudiantest.html')
estudiantes.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 290.54it/s]


Index(['PERIODO', 'ID_ESTUDIANTE', 'IDENTIFICACION', 'PROG', 'FECHA_ADMITIDO',
       'FECHA_DESERTO', 'ESTADO', 'FECHA_GRADO', 'FECHA_NACIMIENTO', 'GENERO',
       'COLEGIO', 'FECHA_GRADO_COLEGIO', 'FECHA_ICFES', 'CIUDAD_COLEGIO'],
      dtype='object')

* (OJO) Se detectan sólo 163 filas.  Posiblemente se requieran más datos.  sin embargo, si el modelo base es MATRICULA y no ESTUDIANTES, se generarían cerca de 9 veces más datos que podrían servir para el piloto.    El modelo por tanto tomaría info de un estudiante que acaba de matricularse durante un nuevo semestre.

* Se detectan 8 PERIODOS desde el 201802 hasta el 202204, lo cual son 4 años.
* Cada estudiante tiene su ID (ok).  
* (OJO) Existe una IDENTIFICACION de cada uno, lo cual no es necesario y es mejor evitar por temas de habeas data.  Será eliminada tanto del excel como del archivo posterior confirmación con ERIK.
* Todos vienen del mismo programa (OK)
* (OJO) hay 85 filas que presentan datos en FECHA_DESERTO.  Frente al total de filas es interesante pues pareciera estar "balanceado", lo que significa que habría cerca de un 50% de deserción!!
* (OJO) frente al estado actual, hay 80 filas activas, 78 inactivas (no cuadra con desercion), y 5 graduados únicamente.
* El género viene balanceado (OK)
* muy pocas instancias de colegios distintos.  Estudiar si hay formas de agruparlos por demografía, geográfia, etc.
* (OJO) Dos campos no traen datos FECHA_GRADO_COLEGIO y FECHA_ICFES
* viene la ciudad del colegio, aunque 117 son de cali y por tanto no hay tanta variabilidad.  Agrupar posiblemente (cali, no-cali).
* NULOS:  de los 163 filas, en FECHA_DESERTO hay datos en 78.  FECHA_GRADO solo en 5.





### **1.2 Admisiones**

In [7]:
admisiones = pd.read_excel('data_original/admisiones.xlsx')
prof = ProfileReport(admisiones)
prof.to_file(output_file='admisiones.html')
admisiones.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 92.63it/s]


Index(['PROGRAMA_CODIGO', 'SEDE_NOMBRE', 'PERIODO_INSCRIPCION',
       'FECHA_INSCRIPCION', 'PROGRAMA_OPCION1_CODIGO', 'NIVELATORIO_INGLES',
       'FECHA_NIVELATORIO_INGLES', 'PRUEBA_DIAGNOSTICA', 'ID_ESTUDIANTE',
       'PERIODO_EXPEDIENTE', 'ACCESO_NOMBRE', 'ESTADO_ACADEMICO',
       'SFDC_ESTADO_ACA', 'CREDITOS_CURSADOS', 'CREDITOS_APROBADOS',
       'FECHA_GRADO', 'RANGO_CREDITOS', 'MEDIO_ENTERO_PROGRAMA',
       'TIPO_SOLICITUD', 'ESTUDIANTE_ARTICULACION', 'PREGUNTA1', 'RESPUESTA1',
       'PREGUNTA2', 'RESPUESTA2', 'PREGUNTA3', 'RESPUESTA3', 'PREGUNTA4',
       'RESPUESTA4', 'PREGUNTA5', 'RESPUESTA5', 'PREGUNTA6', 'RESPUESTA6',
       'EXPEDIENTE_BLOQ_FECHA', 'PROMEDIO_POND_SEM', 'PROMEDIO_POND_ACUM',
       'ULTIMO_PERIODO_MAT', 'CRED_MATRICULADOS', 'ULTIMO_PROCESO_MAT',
       'PILOS_UAO', 'SOLICITUD_PYS_GRADO'],
      dtype='object')

* (OJO) 150 observaciones.  Estudiantes tenía 163.  No todo estudiante tiene info de admisiones.
* todos del mismo programa y misma sede
* NIVELATORIO_INGLES bastante balanceado, con solo 15 missings.
* PRUEBA_DIAGNOSTICA S:89, N:46.  15 missings.   
* ID_ESTUDIANTE:   relación 1:1 entre ADMISIONES y ESTUDIANTES (???)
* ACCESO_NOMBRE:  el tipo de admision (Admision 1er sem, Transferencia interna, Transferencia externa)
* (OJO) ESTADO:  89 activo, 57 inactivo, 6 graduados, 1 Mat (matriculado?).   no cuadra con los datos de estudiante
* (OJO) SFDC_ESTADO_ACA:  no cuadra con ESTADO_ACADEMICO de esta tabla ni con información en ESTUDIANTES.
* (OJO) CREDITOS_CURSADOS:  la cantidad de créditos que ha cursado el estudiante (confirmar). desde 5 hasta 177.    Averiguar datos por semestre (posiblemente en otras tablas)
* (OJO) CREDITOS_APROBADOS:  de 0 a 175.  Interesante.  Ojala conseguir esto por semestre.
* (OJO) FECHA_GRADO.  aparecen 3 instancias.  No cuadra con la info en ESTUDIANTES.
* (OJO) RANGO_CREDITOS.  hay de 10 a 18 y de 16 a 18.  Se sobreponen???
* MEDIO_ENTERO_PROGRAMA: cómo se enteró del programa.  Interesante!!
* (OJO) TIPO_SOLICITUD:  ingreso/transferencia.  Ya cubierta en otra columna.
* (OJO) ESTUDIANTE_ARTICULACIÓN:  NO:140, S:1, NULL: 9  (qué es????   MUY desbalanceado)
* 6 PREGUNTAX / RESPUESTAX.  interesante.
* (OJO) PROMEDIO_POND_SEM:  está en ADMISION? no debería estar en una tabla por semestre?
* (OJO) PROMEDIO_POND_ACUM:  idem al anterior.
* (OJO) CRED_MATRICULADOS: 4 distintos valores.   debería ser por semestre.
* (OJO) ULTIMO_PROCESO_MAT:  ultimo proceso matriculado..  interesante, pero deberia ser mejor por semestre.
* (OJO) PILOS_UAO:  muy desbalanceado.  N: 86, S:1, NULL: 63
* SOLICITUD_PYS_GRADO: interesante.  pero pocas filas lastimósamente. 

IDEA:  centralizar data alrededor del SEMESTRE y no del ESTUDIANTE.  Esto nos aumenta cantidad de filas.   


### **1.3 EstudianteAsignatura**

In [8]:
estudianteasignatura = pd.read_excel('data_original/EstudianteAsignatura.xlsx')
prof = ProfileReport(estudianteasignatura)
prof.to_file(output_file='EstudianteAsignatura.html')
estudianteasignatura.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 486.63it/s]


Index(['PERIODO', 'PERIODO_VERANO', 'ID_ESTUDIANTE', 'COD_DEP', 'COD_ASIG',
       'GRUPO', 'ASIGNATURA-GRUPO', 'CREDITOS', 'CEDULA', 'DEFINITIVA',
       'DEFINITIVA_ALF', 'CANCELADA_VOL', 'CANCELADA_UAO',
       'PERDIDA_INASISTENCIA', 'PERDIDA', 'GANADA'],
      dtype='object')

* 7296 observaciones.  OK.
* PERIODO: viene información desde el 201801 hasta el 202203.  Recordar que 1 y 3 son regulares cierto?
* (OJO) PERIODO_VERANO: todos con False.  Sobra?
* (OJO) ID_ESTUDIANTE:  hay 242 estudiantes distintos, pero en estudiantes hay 163???.   Confirmar si hay estudiantes en esta tabla que no están en la tabla ESTUDIANTES
* COD_DEP:  22 distintos departamentos.  No nulos.
* COD_ASIG:  121 distintas.  No nulos.
* GRUPO: 72 distintos.  No nulos.
* ASIGNATURA-GRUPO:  318 combinaciones.  parece ser la combinación de COD_ASIG y GRUPO.  sobra?
* CREDITOS:  de 2 a 10.  
* (OJO) CEDULA:  203.  No cuadra con estudiantes.  quitar por habeas data.
* DEFINITIVA:  entre 0 y 5, 50 valores.  690 missing = estarán matriculadas y no han terminado?
* (OJO) DEFINITIVA_ALF:  5 valores categóricos.A, R, P, NS, SP.   Buscar la semantica.
* CANCELADA_VOL:  S , N.   11/7285 
* (OJO) CANCELADA_UAO:  S, N.   72/7224    (preguntar semántica de esta variable)
* PERDIDA_INASISTENCIA: S,N. 3/7293.   
* (OJO) PERDIDA:  S,N:  193/7031.    pocos pierden.  ¿porqué la deserción?
* GANADA:  S,N: 860 / 6364



### **1.4 DocentesEstudios**

In [2]:
from pandas_profiling import ProfileReport
import pandas as pd

docentesestudios = pd.read_excel('data_original/DocentesEstudios.xlsx')
prof = ProfileReport(docentesestudios)
prof.to_file(output_file='docentesestudios.html')
docentesestudios.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 296.12it/s]


Index(['EMP_CEDULA', 'INGRESO', 'FEC_ULTIMO_ING_CONTRA_ACT', 'CNOMBRE',
       'GNOMBRE', 'MODALIDAD', 'NACI', 'CLASE', 'EMP_SEXO', 'INI_CONTRATO',
       'FIN_CONTRATO', 'NACIDO_EN', 'CIUDAD_RESIDENCIA', 'NIV_COD', 'NIV_NOM',
       'FECHA_GRADO', 'FECHA_TERM', 'TIPO_NOMINA'],
      dtype='object')

* Sobre DOCENTESESTUDIOS tenemos 121 observaciones con 18 variables (ok)
* (OJO) si en estudiante hay 72 distintos grupos, será que hay más de un docente por grupo?
* EMP_CEDULA:  quitar por habeas data.
* INGRESO:  fecha de ingreso (DD/MM/YYYY)
* (OJO) FEC_ULTIMO_ING_CONTRA_ACT:  Es rea (OJO) y viene en miles ej: 35000.  Tal vez viene en mal formato?
* CNOMBRE: es el tipo de contrato:  DOCENTE HC, PLANTA, PROF_IDIOMAS, PROF_HC_IDIOMAS Y PROF_HC_MAESTRIA
* GNOMBRE: una clasificación mayor del tipo de contrato.  ej: PROF_ASISTENTE_1
* MODALIDAD:  hay 3 valores:  HC, TC, MT 
* NACI:  parece ser la fecha de nacimiento (DD/MM/YYYY)
* (OJO) CLASE: parece repetir modalidad :  Hora Cat. Docente.
* EMP_SEXO: género del empleado.  MAS, FEM
* INI_CONTRATO:  fecha inicio contrato.   DD/MM/YYYY
* FIN_CONTRATO: fecha finald el contrato.  DD/MM/YYYY
* OJO:  verificar si hay varios contratos de un mismo docente.
* NACIDO_EN:  ciudad donde nació el docente.  Categorical.  27.
* CIUDAD_RESIDENCIA:  ciudad donde reside.  categorical.  8.
* NIV_COD y NIV_NOM:  es el id y el nombre del nivel académico:  MAESTRIA, UNIVERSITARIA, DOCTORADO, ESPECIALIZACION, POSTDOC.
* (OJO) FECHA_GRADO: Deber ser la fecha de grado, pero al interior está como real (ej: 36000)
* (OJO) FECHA_TERM:  parece ser una fecha por el formato numérico como el caso anterior, pero no se entiende su significado.
* TIPO_NOMINA:  parece indicar si el pago es normal o integral u HC...  H.CATEDRA, NORMAL, INTEGRAL 




#### **1.4.1 Revisión de si hay más de un contrato por docente**

In [14]:
# miremos si hay más de una fila por docente.
docentesestudios.dtypes
conteo_valores = docentesestudios['EMP_CEDULA'].value_counts()
conteo_mayor_a_1 = conteo_valores[conteo_valores > 1]
conteo_mayor_a_1


EMP_CEDULA
16692463    2
80200393    2
Name: count, dtype: int64

Según la revisión anterior, solo hay 2 docentes que aparecen con dos contratos.  Todos los demás solo aparecen una vez.  CONFIRMAR ESTO.

(OJO) Hablar con ERIK para confirmar si existe historia academica de los docentes, es decir, que se sepa para un mismo docente, cuando pasó de profesional a master, y de ahi a phd, etc.

### **1.5 DocentesAsignaturas**

OJO:  El excel venía con una hoja de cálculo al inicio vacía y por eso no lo leía.  Se la quité para que funcionara.

In [21]:
from pandas_profiling import ProfileReport
import pandas as pd

docenteasignaturas = pd.read_excel('data_original/DocenteAsignaturas.xlsx')
docenteasignaturas.shape
prof = ProfileReport(docenteasignaturas)
prof.to_file(output_file='docenteasignaturas.html')
docenteasignaturas.columns

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 772.72it/s]


Index(['PERIODO', 'PERIODO_VERANO', 'COD_DEP', 'COD_ASIG', 'GRUPO',
       'ASIGNATURA-GRUPO', 'CEDULA'],
      dtype='object')

Vienen 742 instancias.

PERIODO: indica el período desde 201801 (parece) hasta 202203.
(OJO) PERIODO_VERANO:  viene solo Flase.  sobra?
COD_DEP:  código del departamento de la asignatura. 22 valores
COD_ASIG: código de la asignatura.  121 valores.
GRUPO:  código del grupo.  72 valores.
ASIGNATURA-GRUPO: combinatoria de las dos anteriores.
(OJO) CEDULA:  parece ser la cédula del docente..  aunque hay muchos con 2222222..  SOBRA?

